In [1]:
import pandas as pd
import numpy as np

In [20]:
# IMPORTAÇÃO DA BASE DE EMPRESAS INIDÔNEAS E SUSPENSA
# Vamos importar apenas as colunas de CPF/CNPJ e nome do sancionado, pois nos 
# interessa apenas a rotulagem de quais empresas ou sócios estão na lista de 
# inidôneas e suspenas. O nome do sancionado será necessário para o join com a 
# base de sócios, pois nesta o CPF não é completo, sendo necessário join com o nome.
colunas = ['CPF OU CNPJ DO SANCIONADO', 'NOME INFORMADO PELO ÓRGÃO SANCIONADOR',
           'DATA INÍCIO SANÇÃO', 'DATA FINAL SANÇÃO']
inidoneas_e_suspensas_df = pd.read_csv('C:\\DATASETS\Empresas Inidoneas e Suspensas\\' + 
'1-Arquivos Baixados\\20210407_CEIS.csv', sep=';', dtype='str', usecols=colunas, encoding='latin-1')

Processamento/Tratamento de Dados:

In [21]:
# Conferindo campos e tipos
inidoneas_e_suspensas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14832 entries, 0 to 14831
Data columns (total 4 columns):
CPF OU CNPJ DO SANCIONADO                14832 non-null object
NOME INFORMADO PELO ÓRGÃO SANCIONADOR    14832 non-null object
DATA INÍCIO SANÇÃO                       14832 non-null object
DATA FINAL SANÇÃO                        13083 non-null object
dtypes: object(4)
memory usage: 463.6+ KB


In [22]:
# Conferindo campos e valores
inidoneas_e_suspensas_df.describe

<bound method NDFrame.describe of       CPF OU CNPJ DO SANCIONADO  \
0                13459221000174   
1                   70779600800   
2                01276136000191   
3                76258672000163   
4                   37770420806   
...                         ...   
14827            18387009000162   
14828            04163678000146   
14829            06939957000184   
14830            05312963000144   
14831               14399732881   

                   NOME INFORMADO PELO ÓRGÃO SANCIONADOR DATA INÍCIO SANÇÃO  \
0      FLEXNAUTICA EMBARCACOES COMERCIO E SERVICOS EI...         04/06/2019   
1                               OSVALDO PAQUITO DA SILVA         24/01/2019   
2                                  COMERCIAL FERUMA LTDA         14/06/2016   
3      PEDRA AZUL SERVIÇOS DE LIMPEZA E CONSERVAÇÃO LTDA         12/04/2016   
4                                 ARNOLDO LUIZ DE MORAES         09/10/2018   
...                                                  ...               

In [23]:
# Identificando percentual de valores ausentes por coluna:
(inidoneas_e_suspensas_df.isnull().sum() / inidoneas_e_suspensas_df.shape[0] * 100)
                   .sort_values(ascending=False)

DATA FINAL SANÇÃO                        11.792071
DATA INÍCIO SANÇÃO                        0.000000
NOME INFORMADO PELO ÓRGÃO SANCIONADOR     0.000000
CPF OU CNPJ DO SANCIONADO                 0.000000
dtype: float64

### Limpeza e correção dos dados:

##### Limpeza e correção da coluna 'CPF OU CNPJ DO SANCIONADO':

In [24]:
# Conferindo se a coluna 'CPF OU CNPJ DO SANCIONADO' contém realmente apenas
# CPF(11 dígitos) ou CNPJ(14 dígitos). Também fornece informação sobre quantas 
# pessoas físicas e quantas pessoas jurídicas estão presentes na base:
inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'].str.len().value_counts()

11    8426
14    6395
2       11
Name: CPF OU CNPJ DO SANCIONADO, dtype: int64

Verificamos que há algo a ser investigado: 11 registros com valores de tamanho 2 na coluna 'CPF OU CNPJ DO SANCIONADO'. Vamos avaliar:

In [27]:
nomes_sem_cpf_cnpj = inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'].str.len() == 2]
nomes_sem_cpf_cnpj

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR,DATA INÍCIO SANÇÃO,DATA FINAL SANÇÃO
213,-1,LICITATECH INFORMÁTICA LTDA. ME.,02/10/2013,NaN
1323,-1,CARLOS ALBERTO MORAES DE OLIVEIRA,19/11/2012,19/11/2022
1592,-1,HECTOR ERNESTO SANHUEZA CACERES,22/05/2019,22/05/2024
4131,-1,"MODEC INTERNATIONAL, INC",31/03/2021,30/04/2022
5335,-1,Mayara N. Da Silva,26/02/2008,NaN
6281,-1,F. BONTEMPO BRUM PIRES EIRELI ME,30/05/2019,30/05/2024
10997,-1,ROGéLIO BARCHETTI URREA,11/03/2019,11/03/2024
11050,-1,ROGéLIO BARCHETTI URREA,11/03/2019,11/03/2029
12989,-1,GEOAR TURISMO LTDA,30/01/2019,30/01/2024
13920,-1,SR MOREIRA DISTRIBUIDORA LTDA,28/12/2010,NaN


Vamos ver se esses nomes já constam na base, em linhas que possuem cpf/cnpj preenchidos:

In [29]:
inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'].isin(nomes_sem_cpf_cnpj['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'])].sort_values('NOME INFORMADO PELO ÓRGÃO SANCIONADOR')

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR,DATA INÍCIO SANÇÃO,DATA FINAL SANÇÃO
8697,36712230730,CARLOS ALBERTO MORAES DE OLIVEIRA,10/09/2012,10/09/2022
1323,-1,CARLOS ALBERTO MORAES DE OLIVEIRA,19/11/2012,19/11/2022
3325,36712230730,CARLOS ALBERTO MORAES DE OLIVEIRA,10/09/2012,10/09/2022
6281,-1,F. BONTEMPO BRUM PIRES EIRELI ME,30/05/2019,30/05/2024
12989,-1,GEOAR TURISMO LTDA,30/01/2019,30/01/2024
4426,58850785000171,GEOAR TURISMO LTDA,21/10/2019,21/10/2024
1592,-1,HECTOR ERNESTO SANHUEZA CACERES,22/05/2019,22/05/2024
213,-1,LICITATECH INFORMÁTICA LTDA. ME.,02/10/2013,NaN
4131,-1,"MODEC INTERNATIONAL, INC",31/03/2021,30/04/2022
5335,-1,Mayara N. Da Silva,26/02/2008,NaN


Verificamos acima que alguns casos possuem cpf/cnpj de outros registros. Vamos fazer a atribuição deles, e depois remover os que não tiveram cpj/cnpj identificados:

In [54]:
inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'] = np.where(np.logical_and(inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO']=='-1', inidoneas_e_suspensas_df['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'] == 'CARLOS ALBERTO MORAES DE OLIVEIRA'), '36712230730', inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'])
inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'] = np.where(np.logical_and(inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO']=='-1', inidoneas_e_suspensas_df['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'] == 'GEOAR TURISMO LTDA'), '58850785000171', inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'])
inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'] = np.where(np.logical_and(inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO']=='-1', inidoneas_e_suspensas_df['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'] == 'ROGéLIO BARCHETTI URREA'), '05950423844', inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'])

In [56]:
nomes_sem_cpf_cnpj = inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'].str.len() == 2]
nomes_sem_cpf_cnpj

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR,DATA INÍCIO SANÇÃO,DATA FINAL SANÇÃO
213,-1,LICITATECH INFORMÁTICA LTDA. ME.,02/10/2013,NaN
1592,-1,HECTOR ERNESTO SANHUEZA CACERES,22/05/2019,22/05/2024
4131,-1,"MODEC INTERNATIONAL, INC",31/03/2021,30/04/2022
5335,-1,Mayara N. Da Silva,26/02/2008,NaN
6281,-1,F. BONTEMPO BRUM PIRES EIRELI ME,30/05/2019,30/05/2024
13920,-1,SR MOREIRA DISTRIBUIDORA LTDA,28/12/2010,NaN


In [57]:
inidoneas_e_suspensas_df = inidoneas_e_suspensas_df.drop(nomes_sem_cpf_cnpj.index)

In [58]:
# Conferindo a coluna 'CPF OU CNPJ DO SANCIONADO 'após os ajustes:
inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'].str.len().value_counts()

11    8430
14    6396
Name: CPF OU CNPJ DO SANCIONADO, dtype: int64

Fim da limpeza e correção da coluna 'CPF OU CNPJ DO SANCIONADO'

### Análise dos dados:

In [59]:
inidoneas_e_suspensas_df

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR,DATA INÍCIO SANÇÃO,DATA FINAL SANÇÃO
0,13459221000174,FLEXNAUTICA EMBARCACOES COMERCIO E SERVICOS EI...,04/06/2019,04/06/2021
1,70779600800,OSVALDO PAQUITO DA SILVA,24/01/2019,24/01/2023
2,01276136000191,COMERCIAL FERUMA LTDA,14/06/2016,13/06/2021
3,76258672000163,PEDRA AZUL SERVIÇOS DE LIMPEZA E CONSERVAÇÃO LTDA,12/04/2016,11/04/2021
4,37770420806,ARNOLDO LUIZ DE MORAES,09/10/2018,09/10/2023
...,...,...,...,...
14827,18387009000162,INFORGERAIS EIRELI,18/03/2021,NaN
14828,04163678000146,POLO PROFESSIONAL SERVICOS LTDA,04/04/2018,NaN
14829,06939957000184,ELETRICA RAIO DO SOL LTDA,24/04/2018,NaN
14830,05312963000144,COMERCIAL LICITACAO LTDA,27/06/2017,21/06/2023


In [60]:
print(inidoneas_e_suspensas_df['DATA INÍCIO SANÇÃO'].min())
print(inidoneas_e_suspensas_df['DATA INÍCIO SANÇÃO'].max())

01/01/2019
31/12/2020


Datas de início dentro do esperado.

In [65]:
print(inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['DATA FINAL SANÇÃO'].notnull() == True]['DATA FINAL SANÇÃO'].min())
print(inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['DATA FINAL SANÇÃO'].notnull() == True]['DATA FINAL SANÇÃO'].max())

01/01/2019
31/12/2099


In [69]:
inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['DATA FINAL SANÇÃO'].str[-4:] == '2099']

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR,DATA INÍCIO SANÇÃO,DATA FINAL SANÇÃO
14544,02731495000154,CONSERVO BRASILIA EMPRESA DE SEGURANÇA LTDA,22/10/2008,31/12/2099


Já sobre a data final, percebemos um registro que pode ser um erro (31/12/2099). Mas como iremos considerar apenas a existência ou não da empresa na lista e inidôneas e suspensas, então não é preciso nenhum tratamento. Além disso, vimos anteriormente que há linhas com 'DATA FINAL SANÇÃO', e também não faremos tratamento sobre isso.

Vamos remover as colunas de datas, pois serviram apenas para conhecimento e não serão mais necessárias:

In [71]:
inidoneas_e_suspensas_df.drop(columns=['DATA INÍCIO SANÇÃO', 'DATA FINAL SANÇÃO'], inplace=True)

Vamos agora remover duplicidades de CNPJ:

In [96]:
# Primeiro, separar um dataframe só com CNPJ:
inidoneas_e_suspensas_pj_df = inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'].str.len()==14]

In [97]:
# Vamos agrupar por CNPJ e pegar apenas o primeiro nome da empresa:
inidoneas_e_suspensas_pj_df = inidoneas_e_suspensas_pj_df.groupby('CPF OU CNPJ DO SANCIONADO').head(1)

In [98]:
# Vamos testar se deu tudo certo. Primeiro, vamos identificar um cnpj que tenha várias linhas:
inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'].value_counts()

59456353168       41
09175725134       37
10996399000166    25
10699681707       18
08761331708       18
                  ..
13947569000100     1
22711938000152     1
18706523803        1
17653082000176     1
20847823920        1
Name: CPF OU CNPJ DO SANCIONADO, Length: 11749, dtype: int64

In [99]:
# Escolhido o cnpj 10996399000166, vamos ver como ele estava na lista original:
inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO']=='10996399000166']

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR
763,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS - EIRELI
1520,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS - EIRLI
1664,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS EIRELI
1808,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS EIRELLI
1822,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS EIRELI EPP
1966,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS EIRELI EPP
2110,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS - EIRELI - EPP
2254,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS EIRELI - EPP
2398,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS - EIRELI - EPP
2542,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS EIRELI EPP


In [100]:
# Vamos ver como esse mesmo CNPJ ficou na lista consolidada (sem duplicidade):
inidoneas_e_suspensas_pj_df[inidoneas_e_suspensas_pj_df['CPF OU CNPJ DO SANCIONADO']=='10996399000166']

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR
763,10996399000166,SPEED DISTRIBUIDORA E SERVIÇOS - EIRELI


Ou seja, tudo correto na remoção de cnpjs duplicados.

Vamos agora avaliar e remover duplicidades de CPF:

In [115]:
# Primeiro, separar um dataframe só com CPF:
inidoneas_e_suspensas_pf_df = inidoneas_e_suspensas_df[inidoneas_e_suspensas_df['CPF OU CNPJ DO SANCIONADO'].str.len()==11]

In [116]:
# Como não podemos desconsiderar o nome nos casos de linhas de CPF, precisamos remover duplicados usando o método drop, considerando as duas colunas:
print(inidoneas_e_suspensas_pf_df.shape)
inidoneas_e_suspensas_pf_df = inidoneas_e_suspensas_pf_df.drop_duplicates()
print(inidoneas_e_suspensas_pf_df.shape)

(8430, 2)
(6770, 2)


Removidas as duplicidades de cpf e cnpj, vamos unificar os dataframes:

In [119]:
inidoneas_e_suspensas_df = pd.concat([inidoneas_e_suspensas_pj_df, inidoneas_e_suspensas_pf_df])
inidoneas_e_suspensas_df.reset_index(drop=True)

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR
0,13459221000174,FLEXNAUTICA EMBARCACOES COMERCIO E SERVICOS EI...
1,01276136000191,COMERCIAL FERUMA LTDA
2,76258672000163,PEDRA AZUL SERVIÇOS DE LIMPEZA E CONSERVAÇÃO LTDA
3,17015396000143,PRATO CERTO - ALIMENTACAO E NUTRICAO EIRELI
4,21545589000183,R & B PRODUTOS E SERVIÇOS LTDA
...,...,...
11915,53290712087,EDUARDO FLORENTINO PACHECO DA SILVA
11916,15158268134,JOSé VALDéCIO PESSOA
11917,15772497472,UBIRACI BERNADINO GOMES
11918,89196988800,MARCIO EMILIO BARSANTI DE ALMEIDA


Verificamos acima alguns nomes com acentuação e letras minúsculas. Vamos retirar os acentos e colocar tudo em maiúsculo, pois a base de sócios está dessa forma (e precisaremos fazer join de pessoas físicas usando a coluna de nome).

In [125]:
# Definição de uma função para retirar acentos 
# e transformar texto em maiúsculo:

import unicodedata

def strip_accents_and_upper(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text).upper()

s = strip_accents_and_upper('àéêöhello')
print(s)

aeeohello


In [132]:
# Aplicação da função
inidoneas_e_suspensas_df['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'] = 
   inidoneas_e_suspensas_df['NOME INFORMADO PELO ÓRGÃO SANCIONADOR'].map(strip_accents_and_upper)

In [133]:
# Verificando resultados:
inidoneas_e_suspensas_df

,CPF OU CNPJ DO SANCIONADO,NOME INFORMADO PELO ÓRGÃO SANCIONADOR
0,13459221000174,FLEXNAUTICA EMBARCACOES COMERCIO E SERVICOS EI...
2,01276136000191,COMERCIAL FERUMA LTDA
3,76258672000163,PEDRA AZUL SERVICOS DE LIMPEZA E CONSERVACAO LTDA
5,17015396000143,PRATO CERTO - ALIMENTACAO E NUTRICAO EIRELI
6,21545589000183,R & B PRODUTOS E SERVICOS LTDA
...,...,...
14810,53290712087,EDUARDO FLORENTINO PACHECO DA SILVA
14811,15158268134,JOSE VALDECIO PESSOA
14815,15772497472,UBIRACI BERNADINO GOMES
14817,89196988800,MARCIO EMILIO BARSANTI DE ALMEIDA


Por fim, vamos renomear colunas:

In [134]:
inidoneas_e_suspensas_df = inidoneas_e_suspensas_df.rename(columns={"CPF OU CNPJ DO SANCIONADO": "CPF_CNPJ", "NOME INFORMADO PELO ÓRGÃO SANCIONADOR": "NOME"})

Finalizado. Vamos salvar os resultados:

In [135]:
inidoneas_e_suspensas_df.to_csv(r'C:\\DATASETS\Empresas Inidoneas e Suspensas\\inidoneas_e_suspensas.csv', index = False)